In [1]:
# Install a pip package in the current Jupyter kernel
import sys
#!{sys.executable} -m pip install pandas
#!{sys.executable} -m pip install numpy

#import packages
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
# Import data
data_crime = pd.read_csv(r'https://raw.githubusercontent.com/mariusgruenewald/pol_viol/main/data/crime_data_edited.csv')
data_share = pd.read_csv(r'https://raw.githubusercontent.com/mariusgruenewald/pol_viol/main/data/election_data_edited.csv')

In [3]:
data_share.dropna(thresh=4, inplace=True)
data_share.drop_duplicates(subset=['plz', 'city', 'party', 'state', 'city_id', 'year'], inplace=True)

In [4]:
data_crime['date'] = pd.to_datetime(data_crime['date'])
data_crime.rename({"ags":"city_id"}, axis=1, inplace=True)

C:\Users\Marius\AppData\Local\Temp\ipykernel_38504\2509066210.py:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  data_crime['date'] = pd.to_datetime(data_crime['date'])


In [5]:
# Merge crime and candidate statistics with merge over city string
data_crime = data_crime[(data_crime['Land'] == 14) | (data_crime['Land'] == 8) | (data_crime['Land'] == 16) | (data_crime['Land'] == 12)
                       | (data_crime['Land'] == 6)]
data_share['city'] = data_share['city'].str.replace(r'Stadt ', '')
data_share['city'] = data_share['city'].str.replace('Ä', 'A').str.replace('Ö', 'O').str.replace('Ü', 'U')

list_cities_share = data_share.sort_values('city')['city'].unique()
list_cities_crime = data_crime['city'].unique()

data_crime.rename({'Land':'state'}, axis=1, inplace=True)
data_crime['state'] = data_crime['state'].map({16:'TH', 8:'BW', 14:'SN', 12:'BB', 6:'HE'})

In [6]:
data_share = data_share[data_share["year"] != 2011]
data_share["cycle_1"] = data_share["year"]
data_share.loc[data_share["year"] == 2021, 'cycle_1'] = 2019
data_share.loc[data_share["year"] == 2016, 'cycle_1'] = 2014
data_share['cycle_2'] = data_share['cycle_1']
data_share['cycle_3'] = data_share['cycle_1']


Merging over years and cities? How many different cutoffs? Directly on election day, one month before and three months before

Cut-off days for the three states: 05.26.2019, 04.26.2019, 02.26.2019

In [7]:
# creating different cycle variables in crime data to merge over
#
data_crime['cycle_1'] = 2014
data_crime.loc[(data_crime['date'] > dt.datetime(2019, 5, 26, 0, 0)) & (data_crime['state'] != "HE"), 'cycle_1'] = 2019
data_crime.loc[(data_crime['date'] > dt.datetime(2021, 3, 14, 0, 0)) & (data_crime['state'] == "HE"), 'cycle_1'] = 2019

data_crime['cycle_2'] = 2014
data_crime.loc[(data_crime['date'] > dt.datetime(2019, 4, 26, 0, 0)) & (data_crime['state'] != "HE"), 'cycle_2'] = 2019
data_crime.loc[(data_crime['date'] > dt.datetime(2021, 2, 14, 0, 0)) & (data_crime['state'] == "HE"), 'cycle_1'] = 2019

data_crime['cycle_3'] = 2014
data_crime.loc[(data_crime['date'] > dt.datetime(2019, 2, 26, 0, 0)) & (data_crime['state'] != "HE"), 'cycle_3'] = 2019
data_crime.loc[(data_crime['date'] > dt.datetime(2021, 12, 14, 0, 0)) & (data_crime['state'] == "HE"), 'cycle_1'] = 2019


In [8]:
# remove duplicates from data_share
# The following mistake happened: during merger full line and line w/out merger success being kept
# If done before, file will not contain unmatched entities -> necessary for manual inspection
#data_share.drop_duplicates(subset=['city','party','Anzahl Bewerber','darunter Frauen','cycle_1','state'], ignore_index=True, inplace=True)

In [9]:
# Find double cities in data share
# we cannot use plz here, since crime data doesn't have any of that -> merger on plz doesnt work
# list_double = pd.DataFrame()
# for city in data_share['city'].unique():
    
#     data_city = data_share[data_share['city'] == city]
#     for party in data_city['party'].unique():
        
#         data_party = data_city[data_city['party'] == party]
#         for election in data_party['cycle_1'].unique():
            
#             data_election = data_party[data_party['cycle_1'] == election]
            
#             if len(data_election) > 1:
                
#                 # append to dataset of 
#                 list_double = list_double.append(data_election)
                

In [125]:
# drop cities that are not uniquely identifable by cityname
#list_double_cities = data_share[data_share.city.isin(list_double.city)]['city'].unique()
#list_to_drop_crime = data_crime[data_crime.city.isin(list_double.city)]
#data_crime = data_crime[~data_crime.city.isin(list_to_drop_crime.city)]

In [9]:
#data_share.dropna(thresh, inplace=True)

# Generate crime_count variable for easier handling
data_crime['crime_count'] = 1
data_crime.drop('Unnamed: 0', axis=1, inplace=True)
data_crime.dropna(subset=['party'], inplace=True)

In [10]:
data_merged = pd.merge(data_crime, data_share, on=['plz','city', 'cycle_1', 'party', 'state', 'city_id'], how='right', validate='m:1')
data_merged.drop(['Unnamed: 0'], axis=1, inplace=True)
data_merged['crime_count'] = data_merged['crime_count'].fillna(0)
data_merged

,city,date,background,law,suspects,crime,party,state,plz,city_id,...,cycle_3_x,crime_count,Anzahl Bewerber,Prozent Frauen,darunter Frauen,year,Land,city_id_bb,cycle_2_y,cycle_3_y
0,Stuttgart,2019-07-25,Links,STGB-303,0.0,Sachbeschadigung,AfD,BW,70173.0,8111000.0,...,2019.0,1.0,60.0,26.666667,16.0,2019.0,8.0,NaN,2019.0,2019.0
1,Stuttgart,2019-07-25,Links,STGB-187,0.0,Verleumdung ohne Verleumdung ohne sexuelle Gru...,AfD,BW,70173.0,8111000.0,...,2019.0,1.0,60.0,26.666667,16.0,2019.0,8.0,NaN,2019.0,2019.0
2,Stuttgart,2019-12-08,Rechts,STGB-130,1.0,Volksverhetzung,AfD,BW,70173.0,8111000.0,...,2019.0,1.0,60.0,26.666667,16.0,2019.0,8.0,NaN,2019.0,2019.0
3,Stuttgart,2019-08-26,Links,STGB-303,0.0,Sachbeschadigung,AfD,BW,70173.0,8111000.0,...,2019.0,1.0,60.0,26.666667,16.0,2019.0,8.0,NaN,2019.0,2019.0
4,Stuttgart,2019-07-25,Links,STGB-303,0.0,Sachbeschadigung,AfD,BW,70173.0,8111000.0,...,2019.0,1.0,60.0,26.666667,16.0,2019.0,8.0,NaN,2019.0,2019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23636,Zierenberg,NaT,NaN,NaN,NaN,NaN,SPD,HE,34289.0,6633029.0,...,NaN,0.0,32.0,25.000000,8.0,2021.0,6.0,NaN,2019.0,2019.0
23637,Zwingenberg,NaT,NaN,NaN,NaN,NaN,CDU,HE,64673.0,6431022.0,...,NaN,0.0,26.0,26.923077,7.0,2016.0,6.0,NaN,2014.0,2014.0
23638,Zwingenberg,NaT,NaN,NaN,NaN,NaN,SPD,HE,64673.0,6431022.0,...,NaN,0.0,25.0,24.000000,6.0,2016.0,6.0,NaN,2014.0,2014.0
23639,Zwingenberg,NaT,NaN,NaN,NaN,NaN,CDU,HE,64673.0,6431022.0,...,NaN,0.0,35.0,37.142857,13.0,2021.0,6.0,NaN,2019.0,2019.0


In [11]:
data_merged["year"].unique()

array([2019., 2014., 2016., 2021.])

In [12]:
data_merged_2 = data_merged.groupby('city').count()
print(f"Total num of cities w/ crime & target: {len(data_crime['city'].unique())}, \n")
print(f"Total num of cities w/ election: {len(data_share['city'].unique())}, \n")
print(f"Total num of cities w/ crime & election: {len(data_merged_2[data_merged_2['crime'] != 0])}, \n")
print(f"Total num of crime in df: {len(data_merged[~data_merged['crime'].isna()])}")

Total num of cities w/ crime & target: 570, 

Total num of cities w/ election: 2737, 

Total num of cities w/ crime & election: 376, 

Total num of crime in df: 3161


### Generate various measures of crime

In [13]:
# regular summarization across cities and parties
data_crime_base = data_crime.groupby(['city', 'background', 'party', 'cycle_1', 'state'], as_index=False).sum()
data_crime_party = data_crime.groupby(['city', 'party', 'cycle_1', 'state'], as_index=False).sum()
data_crime_lr = data_crime[(data_crime['background'] == 'Links') | (data_crime['background'] == 'Rechts') | (data_crime['background'].isna())] #drop unidentified crime for left-right analysis
data_crime_lr_merge= data_crime_lr.groupby(['city', 'background', 'cycle_1', 'state'], as_index=False).sum()
data_crime_city = data_crime.groupby(['city', 'cycle_1', 'state'], as_index=False).sum()
data_crime_base.rename({'crime_count':'crime_count_party_lr'}, axis=1, inplace=True)
data_crime_party.rename({'crime_count':'crime_count_party'}, axis=1, inplace=True)
data_crime_lr_merge.rename({'crime_count':'crime_count_lr'}, axis=1, inplace=True)
data_crime_city.rename({'crime_count':'crime_count_city'}, axis=1, inplace=True)

C:\Users\Marius\AppData\Local\Temp\ipykernel_38504\3234842448.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data_crime_base = data_crime.groupby(['city', 'background', 'party', 'cycle_1', 'state'], as_index=False).sum()
C:\Users\Marius\AppData\Local\Temp\ipykernel_38504\3234842448.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data_crime_party = data_crime.groupby(['city', 'party', 'cycle_1', 'state'], as_index=False).sum()
C:\Users\Marius\AppData\Local\Temp\ipykernel_38504\3234842448.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, 

In [14]:
# putting all measures into on dataframe. CAUTION: Alsways do drop_duplicates and dropna if other than crime_count used
data_merged = pd.merge(data_merged, data_crime_base[['city', 'background', 'party', 'cycle_1', 'state', 'crime_count_party_lr']], on=['city', 'background', 'party', 'cycle_1', 'state'], how='left', validate='m:1')
data_merged = pd.merge(data_merged, data_crime_party[['city', 'party', 'cycle_1', 'state', 'crime_count_party']], on=['city', 'party', 'cycle_1', 'state'], how='left', validate='m:1')
data_merged = pd.merge(data_merged, data_crime_lr_merge[['city', 'background', 'cycle_1', 'state', 'crime_count_lr']], on=['city', 'background', 'cycle_1', 'state'], how='left', validate='m:1')
data_merged = pd.merge(data_merged, data_crime_city[['city', 'cycle_1', 'state', 'crime_count_city']], on=['city', 'cycle_1', 'state'], how='left', validate='m:1') 

In [15]:
for col in ('crime_count_party_lr','crime_count_party','crime_count_lr','crime_count_city'):
    data_merged[col].fillna(0, inplace=True)
data_merged.tail(5)

,city,date,background,law,suspects,crime,party,state,plz,city_id,...,darunter Frauen,year,Land,city_id_bb,cycle_2_y,cycle_3_y,crime_count_party_lr,crime_count_party,crime_count_lr,crime_count_city
23636,Zierenberg,NaT,NaN,NaN,NaN,NaN,SPD,HE,34289.0,6633029.0,...,8.0,2021.0,6.0,NaN,2019.0,2019.0,0.0,0.0,0.0,0.0
23637,Zwingenberg,NaT,NaN,NaN,NaN,NaN,CDU,HE,64673.0,6431022.0,...,7.0,2016.0,6.0,NaN,2014.0,2014.0,0.0,0.0,0.0,0.0
23638,Zwingenberg,NaT,NaN,NaN,NaN,NaN,SPD,HE,64673.0,6431022.0,...,6.0,2016.0,6.0,NaN,2014.0,2014.0,0.0,0.0,0.0,0.0
23639,Zwingenberg,NaT,NaN,NaN,NaN,NaN,CDU,HE,64673.0,6431022.0,...,13.0,2021.0,6.0,NaN,2019.0,2019.0,0.0,0.0,0.0,0.0
23640,Zwingenberg,NaT,NaN,NaN,NaN,NaN,SPD,HE,64673.0,6431022.0,...,9.0,2021.0,6.0,NaN,2019.0,2019.0,0.0,0.0,0.0,0.0


In [16]:
data_merged['party_lr'] = 'right'
data_merged.loc[(data_merged['party'] == 'SPD') | (data_merged['party'] == 'GRÜNE') | (data_merged['party'] == 'DIE LINKE'), 'party_lr'] = 'left' 

In [17]:
data_share['state'].unique()

array(['BW', 'SN', 'TH', 'BB', 'HE'], dtype=object)

### Generate aggregation of fem_share

In [18]:
data_elec_by_crime_background = data_merged.groupby(['background', 'city', 'cycle_1', 'state', 'plz'], as_index=False).sum()[['background', 'city', 'cycle_1', 'state', 'plz', 'Anzahl Bewerber', 'darunter Frauen']]
data_elec_by_crime_background.rename({'Anzahl Bewerber':'bewerber_hit_by_lr', 'darunter Frauen':'fem_hit_by_lr'}, axis=1, inplace=True)

city_crime = data_merged.groupby(['city', 'cycle_1', 'state', 'plz'], as_index=False).sum()[['city', 'cycle_1', 'state', 'plz', 'Anzahl Bewerber', 'darunter Frauen']]
city_crime.rename({'Anzahl Bewerber':'Bewerber_city', 'darunter Frauen':'Frauen_city'}, axis=1, inplace=True)

data_elec_by_crime_background_party = data_merged.groupby(['background', 'city', 'cycle_1', 'state', 'plz', 'party'], as_index=False).sum()[['background', 'party', 'city', 'cycle_1', 'state', 'plz', 'Anzahl Bewerber', 'darunter Frauen']]
data_elec_by_crime_background_party.rename({'Anzahl Bewerber':'bewerber_party_hit_by_lr', 'darunter Frauen':'fem_party_hit_by_lr'}, axis=1, inplace=True)


C:\Users\Marius\AppData\Local\Temp\ipykernel_38504\4270986936.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data_elec_by_crime_background = data_merged.groupby(['background', 'city', 'cycle_1', 'state', 'plz'], as_index=False).sum()[['background', 'city', 'cycle_1', 'state', 'plz', 'Anzahl Bewerber', 'darunter Frauen']]
C:\Users\Marius\AppData\Local\Temp\ipykernel_38504\4270986936.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  city_crime = data_merged.groupby(['city', 'cycle_1', 'state', 'plz'], as_index=False).sum()[['city', 'cycle_1', 'state', 'plz', 'Anzahl Bewerber', 'darunter Frauen']]
C:\User

In [19]:
data_merged = pd.merge(data_merged, city_crime, on=['city', 'cycle_1', 'state', 'plz'], how='left', validate='m:1')
data_merged = pd.merge(data_merged, data_elec_by_crime_background, on=['background', 'city', 'cycle_1', 'state', 'plz'], how='left', validate='m:1')
data_merged = pd.merge(data_merged, data_elec_by_crime_background_party, on=['background', 'city', 'cycle_1', 'state', 'plz', 'party'], how='left', validate='m:1')

In [20]:
for col in ('bewerber_hit_by_lr','fem_hit_by_lr','Bewerber_city','Frauen_city', 'bewerber_party_hit_by_lr', 'fem_party_hit_by_lr'):
    data_merged[col].fillna(0, inplace=True)
data_merged

,city,date,background,law,suspects,crime,party,state,plz,city_id,...,crime_count_party,crime_count_lr,crime_count_city,party_lr,Bewerber_city,Frauen_city,bewerber_hit_by_lr,fem_hit_by_lr,bewerber_party_hit_by_lr,fem_party_hit_by_lr
0,Stuttgart,2019-07-25,Links,STGB-303,0.0,Sachbeschadigung,AfD,BW,70173.0,8111000.0,...,9.0,11.0,20.0,right,1200.0,434.0,660.0,232.0,420.0,112.0
1,Stuttgart,2019-07-25,Links,STGB-187,0.0,Verleumdung ohne Verleumdung ohne sexuelle Gru...,AfD,BW,70173.0,8111000.0,...,9.0,11.0,20.0,right,1200.0,434.0,660.0,232.0,420.0,112.0
2,Stuttgart,2019-12-08,Rechts,STGB-130,1.0,Volksverhetzung,AfD,BW,70173.0,8111000.0,...,9.0,6.0,20.0,right,1200.0,434.0,240.0,82.0,120.0,32.0
3,Stuttgart,2019-08-26,Links,STGB-303,0.0,Sachbeschadigung,AfD,BW,70173.0,8111000.0,...,9.0,11.0,20.0,right,1200.0,434.0,660.0,232.0,420.0,112.0
4,Stuttgart,2019-07-25,Links,STGB-303,0.0,Sachbeschadigung,AfD,BW,70173.0,8111000.0,...,9.0,11.0,20.0,right,1200.0,434.0,660.0,232.0,420.0,112.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23636,Zierenberg,NaT,NaN,NaN,NaN,NaN,SPD,HE,34289.0,6633029.0,...,0.0,0.0,0.0,left,43.0,10.0,0.0,0.0,0.0,0.0
23637,Zwingenberg,NaT,NaN,NaN,NaN,NaN,CDU,HE,64673.0,6431022.0,...,0.0,0.0,0.0,right,51.0,13.0,0.0,0.0,0.0,0.0
23638,Zwingenberg,NaT,NaN,NaN,NaN,NaN,SPD,HE,64673.0,6431022.0,...,0.0,0.0,0.0,left,51.0,13.0,0.0,0.0,0.0,0.0
23639,Zwingenberg,NaT,NaN,NaN,NaN,NaN,CDU,HE,64673.0,6431022.0,...,0.0,0.0,0.0,right,60.0,22.0,0.0,0.0,0.0,0.0


In [21]:
# p_female works with crime_count_party_lr and crime_count_party
data_merged['p_female'] = data_merged['darunter Frauen']/data_merged['Anzahl Bewerber']
data_merged['p_female_lr'] = data_merged['fem_hit_by_lr']/data_merged['bewerber_hit_by_lr']
data_merged['p_female_city'] = data_merged['Frauen_city']/data_merged['Bewerber_city']
data_merged['p_female_party_lr'] = data_merged['fem_party_hit_by_lr']/data_merged['bewerber_party_hit_by_lr']

### What we have now:
* Panel data set in levels -> always drop duplicates depending on level of analysis

#### Later on, insert control variables here

### What we need:
* Difference in election shares (caution still holds)

In [22]:
## generate differences in party shares
data_unique = data_merged.drop_duplicates(subset=['city', 'state', 'cycle_1', 'party', 'p_female'])
data_panel = pd.DataFrame()
for state in data_unique['state'].unique():
    data_state = data_unique[data_unique['state']==state]
    
    for city in data_state['city'].unique():
        data_city = data_state[data_state['city'] == city]

        for party in data_city['party'].unique():
            data_party = data_city[data_city['party'] == party]

            # drop units with only one party observation
            if len(data_party) > 1:

                diff = data_party[(data_party['cycle_1'] == 2019)]['p_female'].values - data_party[(data_party['cycle_1'] == 2014)]['p_female'].values
                data_merged.loc[(data_merged['party'] == party) & (data_merged['city'] == city), 'p_female_diff'] = diff[0]

In [23]:
# generate differences in city shares
data_unique = data_merged.drop_duplicates(subset=['city', 'state', 'cycle_1', 'p_female_city'])
data_panel = pd.DataFrame()

for state in data_unique['state'].unique():
    data_state = data_unique[data_unique['state']==state]
    
    for city in data_state['city'].unique():
        data_city = data_state[data_state['city'] == city]

        if len(data_city) > 1:         
            diff = data_city[(data_city['cycle_1'] == 2019)]['p_female_city'].values - data_city[(data_city['cycle_1'] == 2014)]['p_female_city'].values
            data_merged.loc[(data_merged['city'] == city), 'p_female_city_diff'] = diff[0]

In [24]:
# generate differences in background shares
data_unique = data_merged.drop_duplicates(subset=['city', 'state', 'cycle_1', 'background', 'p_female_lr'])
data_panel = pd.DataFrame()
for state in data_unique['state'].unique():
    data_state = data_unique[data_unique['state']==state]
    
    for city in data_state['city'].unique():
        data_city = data_state[data_state['city'] == city]
    
        for background in ('Links', 'Rechts'): 
            data_background = data_city[data_city['background'] == background]

            if len(data_background) > 1:
                diff = data_background[(data_background['cycle_1'] == 2019)]['p_female_lr'].values - data_background[(data_background['cycle_1'] == 2014)]['p_female_lr'].values
                data_merged.loc[(data_merged['city'] == city) & (data_merged['background'] == background), 'p_female_lr_diff'] = diff[0]

In [25]:
# generate differences in party-background shares
data_unique = data_merged.drop_duplicates(subset=['city', 'state', 'cycle_1', 'party', 'p_female_party_lr', 'background'])
data_panel = pd.DataFrame()

for state in data_unique['state'].unique():
    data_state = data_unique[data_unique['state']==state]
    
    for city in data_state['city'].unique():
        data_city = data_state[data_state['city'] == city]
        
        for party in data_city['party'].unique():
            data_party = data_city[data_city['party'] == party]

            # drop units with only one party observation
            if len(data_party) > 1:
                for background in ('Links', 'Rechts'):
                    data_b = data_party[data_party['background'] == background]

                    # drop data w/ only one background info
                    if len(data_b) > 1:       

                        diff = data_b[(data_b['cycle_1'] == 2019)]['p_female_party_lr'].values - data_b[(data_b['cycle_1'] == 2014)]['p_female_party_lr'].values
                        data_merged.loc[(data_merged['party'] == party) & (data_merged['city'] == city), 'p_female_party_lr_diff'] = diff[0]

In [26]:
# import other files
data_kreis = pd.read_csv(r'https://raw.githubusercontent.com/mariusgruenewald/pol_viol/main/data/plz_kreis.csv', sep=';')
data_kreis.rename({"PLZ":'plz'}, axis=1, inplace=True)
data_merged = pd.merge(data_merged, data_kreis[['plz','Kreis']], on=['plz'], how='left', validate='m:1')
data_merged.rename({'Kreis':'county'}, axis=1, inplace=True)

In [39]:
# population control data
data_pop = pd.read_csv(r'https://raw.githubusercontent.com/mariusgruenewald/pol_viol/main/data/pop_controls.csv')
data_pop["density"] = data_pop["density"].str.replace(" ", "")
data_pop["density"] = pd.to_numeric(data_pop["density"], errors='coerce')
data_pop["size"] = data_pop["size"].str.replace(",", ".").astype(float)
data_pop.drop(['Unnamed: 0'], axis=1, inplace=True)

In [40]:
data_pop.loc[data_pop["degree_urban"].str.contains('gering besiedelt'), "degree_urban"] = "1"
data_pop.loc[data_pop["degree_urban"].str.contains('mittlere Besiedlungsdichte'), "degree_urban"] = "2"
data_pop.loc[data_pop["degree_urban"].str.contains('dicht besiedelt'), "degree_urban"] = "3"
data_pop["degree_urban"] = data_pop["degree_urban"].astype(float)

In [41]:
data_pop_1 = data_pop.groupby(["plz", "year"]).sum()[["pop", "female", "size"]]
data_pop_2 = data_pop.groupby(["plz", "year"]).mean()[["density", "degree_urban"]]
data_pop_2["degree_urban"] = round(data_pop_2["degree_urban"] )
data_pop_1["share_fem"] = data_pop_1["pop"]/data_pop_1["female"]
data_pop_1.reset_index(drop=False, inplace=True)
data_pop_2.reset_index(drop=False, inplace=True)

In [42]:
data_pop = pd.merge(data_pop_1, data_pop_2, on=["plz", "year"], how='inner', validate='1:1')
data_pop.reset_index(drop=False, inplace=True)

In [43]:
data_merged_2 = pd.merge(data_merged, data_pop, on=['plz', "year"], how='left', validate='m:1')

In [49]:
data_merged_2["day"] = data_merged_2["date"].dt.dayofweek
data_merged_2["year"] = data_merged_2["date"].dt.year

In [50]:
data_merged_2.to_csv(r'C:\Users\Marius\OneDrive - Istituto Universitario Europeo\Documents\pol_viol\data\data_election_crime_merged.csv')